In [1]:
#NVIDIA HARDWARE
! nvidia-smi

Tue Sep  3 19:57:15 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070        On  |   00000000:07:00.0  On |                  N/A |
| 38%   40C    P0             51W /  220W |    1415MiB /   8192MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# CUDA
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Aug_14_10:10:22_PDT_2024
Cuda compilation tools, release 12.6, V12.6.68
Build cuda_12.6.r12.6/compiler.34714021_0


In [3]:
from collections import Counter
from faster_whisper import WhisperModel

# or run on GPU with INT8
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
#model = WhisperModel(model_size, device="cpu", compute_type="int8")

In [4]:
segments, info = model.transcribe(audio="audio_samples/2874774.wav",
                                  language='pt',
                                  beam_size=5)

In [5]:
# Inicializando variáveis para métricas
total_words = 0
word_count = Counter()
magic_words = ["obrigado", "obrigada", "por favor", "desculpa", "desculpe", 
               "por gentileza", "bom dia", "boa tarde", "boa noite", 
               "agradeço", "agradece", "gratidão", "sinto muito", 
               "perdão", "me perdoe", "com licença"]
magic_word_count = Counter({word: 0 for word in magic_words})
total_duration = 0

In [6]:
# Processando a transcrição para calcular métricas
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

    # Dividindo o texto em palavras e convertendo para minúsculas
    words = segment.text.lower().split()
    total_words += len(words)
    word_count.update(words)

    # Contabilizando palavras mágicas
    for word in magic_words:
        magic_word_count[word] += words.count(word)

    # Calculando duração total do áudio em segundos
    total_duration += (segment.end - segment.start)

[0.66s -> 2.16s]  Candinavia Natália, bom dia.
[2.68s -> 4.78s]  Bom dia, Natália. Beatriz da Toto falando.
[4.96s -> 7.04s]  Gostaria de falar que a senhora Silvânia é do TI.
[7.28s -> 9.06s]  Seria ela ainda responsável pelo sistema?
[9.40s -> 10.36s]  Só um momento.
[11.08s -> 11.52s]  Obrigada.
[23.96s -> 24.40s]  Silvânia.
[24.90s -> 27.78s]  Bom dia, senhora Silvânia. Beatriz da Toto falando. Tudo bem?
[27.78s -> 30.50s]  Bom dia, Beatriz. Tudo bom? E você?
[31.10s -> 32.12s]  Também obrigada.
[32.24s -> 34.64s]  A senhora Silvânia, a senhora seria responsável pelo sistema
[34.64s -> 36.94s]  na empresa Scandinavia Veículos, né?
[38.90s -> 39.30s]  Sim.
[39.68s -> 40.12s]  Tá.
[40.58s -> 41.78s]  Meu contato é bem rápido.
[42.20s -> 44.04s]  Eu falo da parte da experiência do cliente.
[44.12s -> 46.40s]  Eu gostaria de saber somente como que está aí a experiência,
[46.60s -> 49.02s]  acompanhar um pouco a percepção que vocês estão tendo.
[49.52s -> 52.10s]  Esse acompanhamento é 

In [7]:
# Calculando métricas finais
words_per_minute = (total_words / total_duration) * 60 if total_duration > 0 else 0
sorted_word_count = word_count.most_common()
magic_word_percentages = {word: (count / total_words) * 100 for word, count in magic_word_count.items() if total_words > 0}


In [8]:
# Exibindo as métricas
print("\nMétricas:")
print(f"Quantidade total de palavras: {total_words}")
print(f"Quantidade de palavras por minuto: {words_per_minute:.2f}")
print("Palavras mais faladas:")
for word, count in sorted_word_count:
    print(f"{word}: {count}")
print("Percentual de palavras mágicas:")
for word, percentage in magic_word_percentages.items():
    print(f"{word}: {percentage:.2f}%")


Métricas:
Quantidade total de palavras: 650
Quantidade de palavras por minuto: 182.04
Palavras mais faladas:
a: 48
senhora: 23
que: 18
o: 17
de: 14
do: 10
tá.: 9
da: 8
eu: 8
é: 7
um: 7
e: 7
né?: 6
como: 6
vocês: 6
então,: 6
bom: 5
gostaria: 5
obrigada.: 5
isso.: 5
gente: 5
tá: 5
para: 5
atendimento: 5
responsável: 4
pelo: 4
tudo: 4
também: 4
empresa: 4
0: 4
você: 4
ou: 4
algum: 4
dia,: 3
seria: 3
na: 3
experiência: 3
vou: 3
são: 3
falando: 3
bom,: 3
então.: 3
10,: 3
qual: 3
teria: 3
não.: 3
os: 3
oito.: 3
em: 3
ao: 3
com: 3
foi: 3
não: 3
entrega?: 3
tem,: 3
beatriz: 2
toto: 2
falando.: 2
falar: 2
ela: 2
só: 2
silvânia.: 2
bem?: 2
bom?: 2
sistema: 2
somente: 2
está: 2
aí: 2
pouco: 2
esse: 2
avalia: 2
10: 2
alguns: 2
pontos: 2
sim,: 2
uma: 2
cnpj: 2
estou: 2
tem: 2
final: 2
é,: 2
por: 2
esses: 2
dez.: 2
mais: 2
peço: 2
tá?: 2
tenha: 2
contato.: 2
nos: 2
suporte,: 2
relação: 2
também.: 2
referente: 2
software?: 2
totalmente,: 2
parcialmente: 2
pode: 2
ser: 2
não,: 2
seis: 2
passando: 2
r